In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

names_list_1 = pd.read_excel("/content/Names List 1.xlsx")
names_list_2 = pd.read_excel("/content/Names List 2.xlsx")


names_list_1['name_state'] = names_list_1['Institution Name'] + ' ' + names_list_1['STABBR']
names_list_2['name_state'] = names_list_2['Institution Name'] + ' ' + names_list_2['State']


vectorizer = TfidfVectorizer()
tfidf_matrix_1 = vectorizer.fit_transform(names_list_1['name_state'])
tfidf_matrix_2 = vectorizer.transform(names_list_2['name_state'])


cosine_sim = cosine_similarity(tfidf_matrix_1, tfidf_matrix_2)
similarity_df = pd.DataFrame(cosine_sim, index=names_list_1['name_state'], columns=names_list_2['name_state'])

matches_df = pd.DataFrame(columns=['Names List 1', 'Names List 2', 'Similarity Score'])


threshold = 0.5
for i, row in similarity_df.iterrows():
    best_match = row.idxmax()
    best_score = row.max()
    if best_score > threshold:
        new_row = pd.DataFrame({
            'Names List 1': [i],
            'Names List 2': [best_match],
            'Similarity Score': [best_score]
        })
        matches_df = pd.concat([matches_df, new_row], ignore_index=True)

matches_df[['Institution Name 1', 'State 1']] = matches_df['Names List 1'].str.rsplit(' ', n=1, expand=True)
matches_df[['Institution Name 2', 'State 2']] = matches_df['Names List 2'].str.rsplit(' ', n=1, expand=True)


matches_df = matches_df.merge(names_list_1[['Institution Name', 'STABBR', 'ID', 'CITY']],
                              left_on=['Institution Name 1', 'State 1'],
                              right_on=['Institution Name', 'STABBR'],
                              how='left')

matches_df.drop(columns=['Institution Name', 'STABBR', 'Names List 1', 'Names List 2'], inplace=True)

matches_df.to_excel("/content/Matched_Institutions.xlsx", index=False)

<ipython-input-4-f23640a8afbc>:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matches_df = pd.concat([matches_df, new_row], ignore_index=True)
